In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import tensorflow as tf

import cv2
import random
import time


from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

from keras.utils import to_categorical
from keras.utils.image_utils import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator


from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from keras.layers import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merging import Concatenate
from keras.layers.core import Lambda, Flatten, Dense

# from keras.initializers import glorot_uniform

from keras.regularizers import l2
from keras import backend as K
#from tensorflow.keras.engine import Layer, InputSpec

from sklearn.utils import shuffle

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os
import sys

In [2]:
food_names = os.listdir("../../data")
food_names

['adhirasam',
 'aloo_gobi',
 'aloo_matar',
 'aloo_methi',
 'aloo_shimla_mirch',
 'aloo_tikki',
 'anarsa',
 'ariselu',
 'bandar_laddu',
 'basundi',
 'bhatura',
 'bhindi_masala',
 'biryani',
 'boondi',
 'butter_chicken',
 'chak_hao_kheer',
 'cham_cham',
 'chana_masala',
 'chapati',
 'chhena_kheeri',
 'chicken_razala',
 'chicken_tikka',
 'chicken_tikka_masala',
 'chikki',
 'daal_baati_churma',
 'daal_puri',
 'dal_makhani',
 'dal_tadka',
 'dharwad_pedha',
 'doodhpak',
 'double_ka_meetha',
 'dum_aloo',
 'gajar_ka_halwa',
 'gavvalu',
 'ghevar',
 'gulab_jamun',
 'imarti',
 'jalebi',
 'kachori',
 'kadai_paneer',
 'kadhi_pakoda',
 'kajjikaya',
 'kakinada_khaja',
 'kalakand',
 'karela_bharta',
 'kofta',
 'kuzhi_paniyaram',
 'lassi',
 'ledikeni',
 'litti_chokha',
 'lyangcha',
 'maach_jhol',
 'makki_di_roti_sarson_da_saag',
 'malapua',
 'misi_roti',
 'misti_doi',
 'modak',
 'mysore_pak',
 'naan',
 'navrattan_korma',
 'palak_paneer',
 'paneer_butter_masala',
 'phirni',
 'pithe',
 'poha',
 'poornalu

In [3]:
def cv2_letterbox_image(image, expected_size):
    ih, iw = image.shape[0:2]
    ew, eh = expected_size
    scale = min(eh / ih, ew / iw)
    nh = int(ih * scale)
    nw = int(iw * scale)
    image = cv2.resize(image, (nw, nh), interpolation=cv2.INTER_CUBIC)
    top = (eh - nh) // 2
    bottom = eh - nh - top
    left = (ew - nw) // 2
    right = ew - nw - left
    new_img = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT)
    return new_img

In [4]:

def get_resized_image(img_shape):
    food_names = os.listdir("../../data")
    path = '../../data'
    class_type = 0
    shapes = []
    for dish in tqdm(food_names,desc='running',file=sys.stdout):
        dish_path = os.path.join(path,dish) + '/'
        for count, filename in enumerate(os.listdir(dish_path)):
            image_path = os.path.join(dish_path, filename)
            image = cv2.imread(image_path)
            shapes.append(image.shape)
            resized_image = cv2_letterbox_image(image,img_shape)
            cv2.imwrite(f"../../data_bak/{dish}/{filename}",resized_image)
# get_resized_image((500,500))

In [5]:
def split_train_test(occupation):
    path = '../../data_bak'
    food_names = os.listdir(path)
    X_train=[]
    y_train = []
    X_test = []
    y_test =[]
    dish_dict = {}
    class_type = 0
    for dish in tqdm(food_names,desc='running',file=sys.stdout):
        dish_dict[dish] = class_type
        dish_path = os.path.join(path,dish) + '/'
        for count, filename in enumerate(os.listdir(dish_path)):
            image_path = os.path.join(dish_path, filename)
            image = cv2.imread(image_path)
            if count < 50*occupation:
                X_train.append(image)
                y_train.append(class_type)
            else:
                X_test.append(image)
                y_test.append(class_type)
        class_type += 1
    return X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = split_train_test(0.8)

running: 100%|██████████| 80/80 [00:39<00:00,  2.02it/s]


In [9]:
def create_pairs(images, labels):
    # initialize two empty lists to hold the (image, image) pairs and
    # labels to indicate if a pair is positive or negative
    random.seed(42)
    pairImages = []
    pairLabels = []

    # calculate the total number of classes present in the dataset
    # and then build a list of indexes for each class label that
    # provides the indexes for all examples with a given label
    numClasses = len(np.unique(y_test))
    classes=np.unique(y_test)
    idx = [np.where(y_test == classes[i]) for i in range(0, numClasses)]

    # loop over all images
    for idxA in range(len(images)):
        # grab the current image and label belonging to the current iteration
        currentImage = images[idxA]
        label = labels[idxA]

        # randomly pick an image that belongs to the *same* class
        # label
        #posId = random.choice(list(np.where(labels == label)))
        posIdx =random.choice([index for index, element in enumerate(labels) if element == label])
        posImage = images[posIdx]

        # prepare a positive pair and update the images and labels
        pairImages.append([currentImage, posImage])
        pairLabels.append([1])

        # grab the indices for each of the class labels *not* equal to
        # the current label and randomly pick an image corresponding
        # to a label *not* equal to the current label
        #negId = random.choice(list(np.where(labels != label)))
        negIdx =random.choice([index for index, element in enumerate(labels) if element != label])
        negImage = images[negIdx]

        # prepare a negative pair of images and update our lists
        pairImages.append([currentImage, negImage])
        pairLabels.append([0])

    return (np.array(pairImages), np.array(pairLabels))

In [10]:
(pairTrain, labelTrain) = create_pairs(X_train, y_train)
(pairTest, labelTest) = create_pairs(X_test, y_test)
len(pairTrain)

6400

In [11]:
len(pairTest[0])

2

In [15]:
IMG_SHAPE = (500,500,3)
def tf_siamese_nn(shape, embedding=64, fineTune=False):
    inputs = tf.keras.layers.Input(shape)
    preprocess_fn = preprocess_input
    base_model = tf.keras.applications.vgg19.VGG19(input_shape=shape, include_top=False,                                               weights='imagenet')

    if fineTune==False:
        base_model.trainable=False
    else:
        base_model.trainable = True
        # Fine-tune from this layer onwards
        fine_tune_at = len(base_model.layers)-int(len(base_model.layers)*.10)
# Freeze all the layers before the `fine_tune_at` layer
        for layer in base_model.layers[:fine_tune_at]:
          layer.trainable =  False
    x=base_model(inputs)
    x=tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs=tf.keras.layers.Dense(embedding)(x)
    model = tf.keras.Model(inputs, outputs)

    return model

model1=tf_siamese_nn(IMG_SHAPE, 64, True)
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 500, 500, 3)]     0         
                                                                 
 vgg19 (Functional)          (None, 15, 15, 512)       20024384  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
Total params: 20,057,216
Trainable params: 2,392,640
Non-trainable params: 17,664,576
_________________________________________________________________


In [16]:
def euclidean_distance(vectors):
    # unpack the vectors into separate lists
    (featsA, featsB) = vectors
    # compute the sum of squared distances between the vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1,keepdims=True)
    # return the euclidean distance between the vectors
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

In [17]:
img1 = tf.keras.layers.Input(shape=IMG_SHAPE)
img2 =  tf.keras.layers.Input( shape=IMG_SHAPE)
featureExtractor = tf_siamese_nn(IMG_SHAPE)
featsA = featureExtractor(img1)
featsB = featureExtractor(img2)

In [18]:
distance = tf.keras.layers.Lambda(euclidean_distance)([featsA, featsB])
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(distance)
model = tf.keras.Model(inputs=[img1, img2], outputs=outputs)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
history = model.fit([pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:], batch_size=1, epochs=20)

MemoryError: Unable to allocate 4.47 GiB for an array with shape (6400, 500, 500, 3) and data type uint8